In [1]:
import os
import sys
import pickle

import cv2
import numpy as np
import neuroglancer
from tqdm import tqdm
from cloudvolume import CloudVolume

sys.path.append(os.path.join(os.getcwd(), '../'))
from utilities.sqlcontroller import SqlController

Connecting dklab@db.dk.ucsd.edu:3306


In [2]:
animal = 'MD589'
downsample_factor = 16

# OUTPUT_DIR_PATH = os.path.join(os.path.expanduser('~'))
OUTPUT_DIR_PATH = os.path.join('./')
IMAGE_DIR_PATH = f'/net/birdstore/Active_Atlas_Data/data_root/pipeline_data/{animal}/preps/CH1/full'
# NG_DIR_PATH = os.path.expanduser('~') 
NG_DIR_PATH = os.path.join('/net/birdstore/Active_Atlas_Data/data_root/pipeline_data/', animal, 'neuroglancer_data')

sqlController = SqlController(animal)
resolution = sqlController.scan_run.resolution
aligned_shape = np.array((sqlController.scan_run.width, sqlController.scan_run.height))
num_section = len(os.listdir(IMAGE_DIR_PATH))

downsampled_aligned_shape = np.round(aligned_shape / downsample_factor).astype(int)
scales = np.array([resolution * downsample_factor, resolution * downsample_factor, 20]) * 1000

known_foundation_structures = ['MVePC', 'DTgP', 'VTA', 'Li', 'Op', 'Sp5C', 'RPC', 'MVeMC', 'APT', 'IPR', 
                               'Cb', 'pc', 'Amb', 'SolIM', 'Pr5VL', 'IPC', '8n', 'MPB', 'Pr5', 'SNR', 
                               'DRD', 'PBG', '10N', 'VTg', 'R', 'IF', 'RR', 'LDTg', '5TT', 'Bar', 
                               'Tz', 'IO', 'Cu', 'SuVe', '12N', '6N', 'PTg', 'Sp5I', 'SNC', 'MnR', 
                               'RtTg', 'Gr', 'ECu', 'DTgC', '4N', 'IPA', '3N', '7N', 'LC', '7n', 
                               'SC', 'LPB', 'EW', 'Pr5DM', 'VCA', '5N', 'Dk', 'DTg', 'LVe', 'SpVe', 
                               'MVe', 'LSO', 'InC', 'IC', 'Sp5O', 'DC', 'Pn', 'LRt', 'RMC', 'PF', 
                               'VCP', 'CnF', 'Sol', 'IPL', 'X', 'AP', 'MiTg', 'DRI', 'RPF', 'VLL']
db_structures = sqlController.get_structures_dict()
non_db_structures = [structure for structure in known_foundation_structures if structure not in db_structures.keys()]

def get_structure_number(structure):
    if structure in db_structures:
        color = db_structures[structure][1]
    elif structure in non_db_structures:
        color = len(db_structures) + non_db_structures.index(structure) + 1
    else:
        color = 255
    return color

In [3]:
def draw_numpy(section_structure_polygons, section_start, section_end, draw_lines=True):
    volume = np.zeros((downsampled_aligned_shape[1], downsampled_aligned_shape[0], section_end - section_start), dtype=np.uint8)
    for section in tqdm(range(section_start, section_end)):
        if section in section_structure_polygons:
            template = np.zeros((downsampled_aligned_shape[1], downsampled_aligned_shape[0]), dtype=np.uint8)
            for structure in section_structure_polygons[section]:
                polygons = section_structure_polygons[section][structure]
                for polygon in polygons:
                    color = get_structure_number(structure)
                    
                    if draw_lines:
                        cv2.polylines(template, [polygon.astype(np.int32)], True, color, 1)
                    else:
                        for point in polygon:
                            cv2.circle(template, tuple(point.astype(np.int32)), 0, color, -1)

            volume[:, :, section - section_start - 1] = template
        
    volume = np.swapaxes(volume, 0, 1)
    return volume

def get_ranges(start, end, interval):
    ranges = []
    
    cur = start
    while cur + interval <= end:
        next = cur + interval
        ranges.append((cur, next))
        cur = next
    ranges.append((cur, end))
    return ranges

In [8]:
draw_lines = False
polygons_path = os.path.join(OUTPUT_DIR_PATH, f'MD589_CVAT_down{downsample_factor}.pkl')

with open(polygons_path, 'rb') as file:
    section_structure_polygons = pickle.load(file)
volume = draw_numpy(section_structure_polygons, 0, num_section, draw_lines=draw_lines)

100%|██████████| 447/447 [00:01<00:00, 242.31it/s]


In [9]:
precompute_path = os.path.join(NG_DIR_PATH, f'{animal}_down{downsample_factor}_{["dots", "lines"][draw_lines]}_annotations')
layer_type = 'segmentation'
offset = [0, 0, 0]

cloudpath = f'file://{precompute_path}'
info = CloudVolume.create_new_info(
    num_channels = volume.shape[3] if len(volume.shape) > 3 else 1,
    layer_type = layer_type,
    data_type = str(volume.dtype), # Channel images might be 'uint8'
    encoding = 'raw', # raw, jpeg, compressed_segmentation, fpzip, kempressed
    resolution = scales, # Voxel scaling, units are in nanometers
    voxel_offset = offset, # x,y,z offset in voxels from the origin
    chunk_size = [64, 64, 64], # units are voxels
    volume_size = volume.shape[:3], # e.g. a cubic millimeter dataset
)
vol = CloudVolume(cloudpath, mip=0, info=info, compress=False)
vol.commit_info()
vol[:, :, :] = volume[:, :, :]

Uploading: 100%|██████████| 9632/9632 [00:52<00:00, 181.92it/s]


In [7]:
# Run this cell if you want to preview the numpy array before converting to precomputed format
import neuroglancer
viewer = neuroglancer.Viewer()
print(viewer)

all_volume_layer = neuroglancer.SegmentationLayer(
    source = neuroglancer.LocalVolume(
        data=volume, 
        dimensions=neuroglancer.CoordinateSpace(names=['x', 'y', 'z'], units='nm', scales=scales), 
        voxel_offset=offset
    ),
)

with viewer.txn() as s:
    s.layers.clear()
    s.layers['all'] = all_volume_layer

http://127.0.0.1:39695/v/577e1d5548425b858e5db2f13724165d84adc392/
